In [1]:
import synapseclient
from synapseclient import Project, File, Folder
from synapseclient import Schema, Column, Table, Row, RowSet, as_table_columns
import pandas as pd

In [2]:
syn = synapseclient.Synapse()
syn.login(authToken=[redacted])

Welcome, Victor Baham!



#### Query FreshMicro staging FileViews

In [17]:
fm_iso_query = syn.tableQuery(f"SELECT * FROM {syn.get('syn64363483').id}")
fm_rna_query = syn.tableQuery(f"SELECT * FROM {syn.get('syn64367073').id}")
fm_scrna_query = syn.tableQuery(f"SELECT * FROM {syn.get('syn64367074').id}")

fm_iso = pd.read_csv(fm_iso_query.filepath)
fm_rna = pd.read_csv(fm_rna_query.filepath)
fm_scrna = pd.read_csv(fm_scrna_query.filepath)

Downloaded syn64363483 to /home/jovyan/.synapseCache/68/150677068/SYNAPSE_TABLE_QUERY_150677068.csv


Downloaded syn64367073 to /home/jovyan/.synapseCache/70/150677070/SYNAPSE_TABLE_QUERY_150677070.csv


Downloaded syn64367074 to /home/jovyan/.synapseCache/72/150677072/SYNAPSE_TABLE_QUERY_150677072.csv


#### Get the columns we will need to keep from the joined DataFrames (the columns are the same in each one, so we can arbitrarily select one of them)

In [8]:
fm_iso.columns

Index(['ROW_ID', 'ROW_VERSION', 'ROW_ETAG', 'id', 'name', 'sex', 'assay',
       'grant', 'organ', 'study', 'tissue', 'runType', 'species', 'cellType',
       'dataType', 'platform', 'consortium', 'fileFormat', 'readLength',
       'specimenID', 'dataSubtype', 'libraryPrep', 'individualID',
       'resourceType', 'isModelSystem', 'isMultiSpecimen',
       'nucleicAcidSource'],
      dtype='object')

#### Map filename to `individualID` for ISOseq files

`individualID` in all cases ends in '_' + 3 digits, so this function can be reused

In [74]:
def dash_three(x):
    dt_idx = x.index('_')
    return x[0:dt_idx+4]

iso_to_id = dict(zip(fm_iso['name'], fm_iso['name'].apply(dash_three)))

#### Map filename to `individualID` for RNAseq files

In [75]:
rna_to_id = dict(zip(fm_rna['name'], fm_rna['name'].apply(dash_three)))

#### Map filename to `individualID` for scRNAseq files

In [76]:
scrna_to_id = dict(zip(fm_scrna['name'], fm_scrna['name'].apply(dash_three)))

#### Turn metadata files to DataFrames

In [150]:
fm_indm = pd.read_csv(syn.get('syn64363368').path)
fm_biom = pd.read_csv(syn.get('syn64364077').path)
fm_rnam = pd.read_csv(syn.get('syn54090267').path)
fm_scrnam = pd.read_csv(syn.get('syn64330569').path)
fm_isom = pd.read_csv(syn.get('syn64367034').path)

#### Fill in `individualID` column for each df

In [83]:
fm_iso['individualID'] = fm_iso['name'].map(iso_to_id)
fm_rna['individualID'] = fm_rna['name'].map(rna_to_id)
fm_scrna['individualID'] = fm_scrna['name'].map(scrna_to_id)

#### **NOTE**: Not currently possible to proceed without mappings between filename and specimenID

#### Fill in the rest of the columns in `fm_iso`

#### Fill in the rest of the columns in `fm_rna`

#### Fill in the rest of the columns in `fm_scrna`